In [ ]:
#USEFUL LINKS
# LINK TO REPLACE WORD IN SENTENCE
# https://www.programiz.com/python-programming/methods/string/replace

In [2]:
import pickle
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Model
from keras.layers import Dense
from keras.layers import Embedding
from tensorflow.keras.layers import LSTM


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pickle.load(open('/content/drive/MyDrive/train.pkl','rb'))

In [4]:
embedding_matrix = pickle.load(open('/content/drive/MyDrive/embedding_matrix.pkl','rb'))


In [5]:
tknizer_ERRONEOUS_SENTENCE = pickle.load(open('/content/drive/MyDrive/tknizer_ERRONEOUS_SENTENCE.pkl','rb'))
tknizer_CORRECT_SENTENCE = pickle.load(open('/content/drive/MyDrive/tknizer_CORRECT_SENTENCE.pkl','rb'))

In [6]:

vocab_size_CORRECT_SENTENCE=len(tknizer_CORRECT_SENTENCE.word_index.keys())
print(vocab_size_CORRECT_SENTENCE)
vocab_size_ERRONEOUS_SENTENCE=len(tknizer_ERRONEOUS_SENTENCE.word_index.keys())
print(vocab_size_ERRONEOUS_SENTENCE)

34411
86321


In [7]:
embedding_matrix = pickle.load(open('/content/drive/MyDrive/embedding_matrix.pkl','rb'))


In [8]:
vanilla = tf.keras.models.load_model('/content/drive/MyDrive/save_model/enc_dec')
vanilla.summary()

Model: "vanilla_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (Encoder)          multiple                  9004468   
_________________________________________________________________
decoder_1 (Decoder)          multiple                  3816468   
_________________________________________________________________
dense_1 (Dense)              multiple                  8868556   
Total params: 21,689,492
Trainable params: 21,689,492
Non-trainable params: 0
_________________________________________________________________


In [9]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, enc_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder", input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    def call(self, input_sentances, training=True):
        input_embedd                        = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c


In [10]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, dec_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.input_length = input_length
    
    def build(self, input_shape):
        # we are using embedding_matrix weights and not training the embedding layer
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix],input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")
        
    def call(self, target_sentances, state_h, state_c):
        target_embedd           = self.embedding(target_sentances)
        lstm_output, _,_        = self.lstm(target_embedd, initial_state=[state_h, state_c])
        return lstm_output

In [11]:
class vanilla_model(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__() 
        self.encoder = Encoder(vocab_size=vocab_size_ERRONEOUS_SENTENCE + 1, embedding_dim=100, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=vocab_size_CORRECT_SENTENCE + 1, embedding_dim=100, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        
        
    def call(self, data):
        input,output = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input)
        decoder_output                       = self.decoder(output, encoder_h, encoder_c)
        output                               = self.dense(decoder_output)
        return output

In [12]:
vanilla = vanilla_model(encoder_inputs_length=16,decoder_inputs_length=16,output_vocab_size=vocab_size_CORRECT_SENTENCE)

In [13]:
def inference(enc_inp,dec_inp):
        
    translation=""

    e_input=[]
    for i in enc_inp.split():
        if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

    #e_input = pad_sequences(e_input, maxlen=16, padding='post')


    e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

    #there is no onestep decoder in this thing, so I have to use the decoder input to predict output

    #decoder input
    d_input=[]
    for i in dec_inp.split():
        if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
            d_input.append(0)
        else:
            d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

    #d_input = pad_sequences(d_input, maxlen=16, padding='post')

    prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

    for word in prediction[0]:
        word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
        if word == "<end>":
            break
    translation += word + " "
    
    return translation
a = 'tihs is bad'
b =  '<start> tihs is bad'
pred = inference(a,b)
pred

'wallace '

In [14]:
a = 'Tihs is bad and we are good'
b = '<start> ' + a
pred = inference(a,b)
pred , a , b

('barnsley ',
 'Tihs is bad and we are good',
 '<start> Tihs is bad and we are good')